In [3]:
#%pip install -q -U google-generativeai easyocr opencv-python-headless fastapi uvicorn python-multipart pyngrok nest_asyncio

In [4]:
import google.generativeai as genai
import easyocr
import cv2
import numpy as np
import json
import os
import uvicorn
import nest_asyncio
from fastapi import FastAPI, File, UploadFile, HTTPException
from pyngrok import ngrok
from PIL import Image
import io

In [ ]:
# ---------------- CONFIGURATION ----------------
# Définition de la clé API Google pour l'accès à Gemini
print("🔐 Initialisation de la clé API Google...")
os.environ['GOOGLE_API_KEY'] = "VOTRE_CLE_API_ICI" 
print("✅ Clé API Google définie dans les variables d'environnement.")

# Token d'authentification Ngrok (permet d'exposer un service local)
print("🌐 Configuration du token Ngrok...")
NGROK_AUTH_TOKEN = "VOTRE_TOKEN_ICI"  # Créez un compte gratuit sur ngrok.com
print("✅ Token Ngrok configuré.")

# Configuration du SDK Gemini avec la clé API
print("⚙️ Configuration du SDK Gemini...")
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
print("✅ SDK Gemini configuré.")

# Initialisation du modèle Gemini utilisé pour la génération
print("🤖 Chargement du modèle Gemini (gemini-2.5-flash-lite)...")
model = genai.GenerativeModel('gemini-2.5-flash-lite')
print("✅ Modèle Gemini prêt.")

# Initialisation OCR (Chargé une seule fois au démarrage)
print("⏳ Chargement du modèle OCR EasyOCR (langues : EN, FR | GPU activé)...")
reader = easyocr.Reader(['en', 'fr'], gpu=True)
print("✅ Modèle OCR chargé.")


🔐 Initialisation de la clé API Google...
✅ Clé API Google définie dans les variables d'environnement.
🌐 Configuration du token Ngrok...
✅ Token Ngrok configuré.
⚙️ Configuration du SDK Gemini...
✅ SDK Gemini configuré.
🤖 Chargement du modèle Gemini (gemini-2.5-flash-lite)...
✅ Modèle Gemini prêt.
⏳ Chargement du modèle OCR EasyOCR (langues : EN, FR | GPU activé)...
✅ Modèle OCR chargé.


In [6]:
# ---------------- LOGIQUE ---------------- #

def clean_json_text(text):
    """
    🔹 Nettoie le texte brut pour ne garder que le JSON.
    Supprime les balises ```json et ``` éventuelles.
    """
    text = text.replace("```json", "").replace("```", "").strip()
    print("🧹 Texte nettoyé pour JSON :", text[:100], "...")  # log des 100 premiers caractères
    return text


def process_image(img_cv):
    """
    🔹 Traite une image :
    1️⃣ OCR pour extraire le texte
    2️⃣ Génération d'un JSON via Gemini selon le type document (Facture / Vin)
    """
    # A. OCR
    print("📸 Début OCR sur l'image...")
    result_ocr = reader.readtext(img_cv, detail=0)
    raw_text = " ".join(result_ocr)
    print("✅ OCR terminé. Texte extrait :", raw_text[:100], "...")  # log partiel pour lisibilité

    # B. PROMPT GEMINI
    prompt = f"""
    Tu es un assistant expert en extraction de données.
    Analyse ce texte brut OCR : "{raw_text}"
    
    Détermine si c'est 'Facture' ou 'Vin'.
    
    Si FACTURE, extrais (JSON) :
    - type: "Facture"
    - date (JJ/MM/AAAA)
    - vendeur
    - montant_total
    - numero_facture
    
    Si VIN, extrais (JSON) :
    - type: "Vin"
    - nom
    - millesime
    - appellation
    - degre_alcool
    
    Réponds UNIQUEMENT en JSON valide.
    """
    print("✏️ Prompt envoyé à Gemini :", prompt[:200], "...")  # log partiel du prompt

    # Envoi du prompt au modèle Gemini
    print("🤖 Génération du JSON via Gemini...")
    response = model.generate_content(prompt)
    print("✅ Réponse Gemini reçue :", response.text[:200], "...")  # log partiel

    # Tentative de parsing JSON
    try:
        parsed_json = json.loads(clean_json_text(response.text))
        print("✅ Parsing JSON réussi :", parsed_json)
        return parsed_json
    except Exception as e:
        print("⚠️ Erreur parsing JSON :", e)
        return {"error": "Echec du parsing JSON", "raw_ai": response.text}


In [7]:
# ---------------- DÉFINITION DE L'API (FastAPI) ----------------

# Création de l'application FastAPI
print("🚀 Initialisation de l'API FastAPI...")
app = FastAPI()
print("✅ API FastAPI prête.")


# Endpoint racine pour vérifier si l'API est en ligne
@app.get("/")
def home():
    print("🏠 Endpoint '/' appelé")
    return {"status": "L'API OCR+Gemini est en ligne !"}


# Endpoint principal pour analyser une image
@app.post("/analyze")
async def analyze_endpoint(file: UploadFile = File(...)):
    print("📩 Endpoint '/analyze' appelé")

    # Vérification du type de fichier
    print("🔍 Vérification du type de fichier :", file.content_type)
    if not file.content_type.startswith('image/'):
        print("❌ Fichier non image reçu")
        raise HTTPException(status_code=400, detail="Le fichier doit être une image.")
    print("✅ Fichier image validé")

    try:
        # Lecture de l'image en mémoire
        print("🖼️ Lecture du fichier image...")
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        img_cv = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        print("✅ Image chargée en mémoire pour traitement")

        # Traitement de l'image via OCR + Gemini
        print("⚙️ Traitement de l'image avec OCR + Gemini...")
        result = process_image(img_cv)
        print("✅ Traitement terminé. Résultat :", result)
        return result

    except Exception as e:
        print("⚠️ Erreur lors du traitement :", str(e))
        return {"error": str(e)}


🚀 Initialisation de l'API FastAPI...
✅ API FastAPI prête.


In [8]:
# ---------------- LANCEMENT DU SERVEUR ----------------
print("🔄 Application du patch nest_asyncio pour Jupyter/async...")
nest_asyncio.apply()
print("✅ nest_asyncio appliqué")

# Configuration du token Ngrok pour exposer localement le serveur
print("🌐 Configuration du token Ngrok...")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ Token Ngrok configuré")

# Nettoyage de toute connexion Ngrok existante
print("🧹 Fermeture des tunnels Ngrok existants...")
ngrok.kill()
print("✅ Tunnels Ngrok fermés")

# Création d'un nouveau tunnel Ngrok sur le port 8000
print("🔌 Création d'un tunnel Ngrok sur le port 8000...")
tunnel = ngrok.connect(8000)
print(f"✅ Tunnel Ngrok créé. URL publique : {tunnel.public_url}")

# Affichage des informations d'accès à l'API
print(f"\n🚀 VOTRE API EST ACCESSIBLE ICI : {tunnel.public_url}")
print(f"👉 Endpoint d'analyse : {tunnel.public_url}/analyze (Méthode POST)")
print(f"👉 Documentation auto : {tunnel.public_url}/docs")

# Configuration et lancement du serveur Uvicorn
print("⚙️ Configuration du serveur Uvicorn...")
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
print("🚀 Lancement du serveur Uvicorn (CTRL+C pour arrêter)...")
await server.serve()


🔄 Application du patch nest_asyncio pour Jupyter/async...
✅ nest_asyncio appliqué
🌐 Configuration du token Ngrok...
✅ Token Ngrok configuré
🧹 Fermeture des tunnels Ngrok existants...
✅ Tunnels Ngrok fermés
🔌 Création d'un tunnel Ngrok sur le port 8000...
✅ Tunnel Ngrok créé. URL publique : https://2b8b2d996e55.ngrok-free.app

🚀 VOTRE API EST ACCESSIBLE ICI : https://2b8b2d996e55.ngrok-free.app
👉 Endpoint d'analyse : https://2b8b2d996e55.ngrok-free.app/analyze (Méthode POST)
👉 Documentation auto : https://2b8b2d996e55.ngrok-free.app/docs
⚙️ Configuration du serveur Uvicorn...


INFO:     Started server process [91572]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🚀 Lancement du serveur Uvicorn (CTRL+C pour arrêter)...
INFO:     89.30.29.68:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     89.30.29.68:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [91572]
